In [ ]:
import logging
from utility import add_project_root_to_path

logging.basicConfig(level=logging.ERROR)

add_project_root_to_path()

In [ ]:
# Building experiments configs

from experiments.configs import DATA_SOURCE_BY_ALIAS, DEFAULT_UNINFORMED_USERS_CONFIG
from experiments.experiment import Experiment
from user.informed_user import InformedUser

from fee_algorithm.fixed_fee import FixedFee
from fee_algorithm.discrete_fee_perfect_oracle import DiscreteFeePerfectOracle
from fee_algorithm.based_on_trade_count_fee import BasedOnTradeCountFee
from fee_algorithm.adaptive_fee_based_on_block_price_move import AdaptiveBasedOnPreviousBlockPriceMoveFee
from experiments.run_multiple_experiments import run_multiple_experiments, get_experiment_key
from fee_algorithm.dynamic_fee_amm_for_amm import AMMforAMMfee
import pandas as pd

from copy import deepcopy

fee_algos_to_consider = {
    "fixed_fee": FixedFee(exchange_fee_rate=0.003), # 30 bps
    "based_on_trade_count_fee": BasedOnTradeCountFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "adaptive_based_on_previous_block_price_move_fee": AdaptiveBasedOnPreviousBlockPriceMoveFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "discrete_fee_perfect_oracle": DiscreteFeePerfectOracle(fee_rate_in_arbitrage_direction=0.0045, fee_rate_in_non_arbitrage_direction=0.0015), # 45/15 bps
    "AMM_fee": AMMforAMMfee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003),
}

In [ ]:
df_stable_not_stable = pd.read_csv('df_stable_not_stable.csv')
df_stable_stable = pd.read_csv('df_stable_stable.csv')
df_not_stable_not_stable = pd.read_csv('df_not_stable_not_stable.csv')

In [ ]:
df_not_stable_not_stable.head()

In [ ]:
df_not_stable_not_stable['start_time'] = pd.to_datetime(df_not_stable_not_stable['start_time'])
df_not_stable_not_stable['end_time'] = pd.to_datetime(df_not_stable_not_stable['end_time'])

df_stable_not_stable['start_time'] = pd.to_datetime(df_stable_not_stable['start_time'])
df_stable_not_stable['end_time'] = pd.to_datetime(df_stable_not_stable['end_time'])

df_stable_stable['start_time'] = pd.to_datetime(df_stable_stable['start_time'])
df_stable_stable['end_time'] = pd.to_datetime(df_stable_stable['end_time'])

In [ ]:
df_stable_not_stable['pair_description'] = 'stable_not_stable'
df_stable_stable['pair_description'] = 'stable_stable'
df_not_stable_not_stable['pair_description'] = 'not_stable_not_stable'

df_all_pairs = pd.concat(
    [
    df_stable_not_stable, 
    df_stable_stable, 
    df_not_stable_not_stable
    ], 
    ignore_index=True,
)

In [ ]:
df_all_pairs

In [ ]:
print(f"Number of segments to save: {df_all_pairs[["Period", "Pair"]].drop_duplicates().shape[0]}")

### Stable/Non-Stable Pairs

In [ ]:
from experiments.experiment import HistoricalDataDescription

experiment_configs = {}

for i, row in df_all_pairs.iterrows():
    data_source = HistoricalDataDescription(
        start_time=row['start_time'],
        end_time=row['end_time'], 
        A_symbol=row['A_symbol'], 
        B_symbol=row['B_symbol'],
        cache_data=True,
    )
    for fee_algo_alias, fee_algo in fee_algos_to_consider.items():
        # if (len(experiment_configs) > 200):
        #     break
        experiment_configs[
            get_experiment_key(
                {
                    "period_alias": str(row['Market_type']),
                    "fee_algorithm": fee_algo_alias,
                    "pair_description": str(row["pair_description"]),
                    "A_symbol": str(row["A_symbol"]),
                    "B_symbol": str(row["B_symbol"]),
                    "start_time": str(row["start_time"]),
                    "end_time": str(row["end_time"]),
                }
            )
        ] = Experiment(
            data=data_source,
            fee_algorithm=deepcopy(fee_algo),
            uninformed_users=deepcopy(DEFAULT_UNINFORMED_USERS_CONFIG),
            informed_user=InformedUser(),
        )

In [ ]:
from pprint import pprint

first_key = list(experiment_configs.keys())[0]
pprint(experiment_configs[first_key])

In [ ]:
print(
    f"Total experiments to run: {len(experiment_configs)}"
)

In [ ]:
experiment_results = run_multiple_experiments(
    experiments=experiment_configs,
    return_intermediate_results=False,
    parallel=True,
    max_workers=8, 
    # parallel=False,
)

In [ ]:
from visualizations.compare_fee_algoritms import get_experiments_summary_by_description

combined_df = get_experiments_summary_by_description(
    experiment_results,
).sort_values(by=["period_alias", "fee_algorithm", "pair_description"])

display(combined_df)